In [72]:
from dotenv import load_dotenv
import os
# from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI

In [8]:
load_dotenv()

True

In [9]:
google_api_key = os.getenv("GOOGLE_API_KEY")


In [83]:
# llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)
# Create LLM class
llm = ChatGoogleGenerativeAI(
    model= "gemini-2.0-flash",
    temperature=1.0,
    max_retries=2,
    google_api_key=google_api_key,
)

In [84]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [122]:
# creaing a tool to multiply two numbers
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    # print("hello i am a tool")
    return a * b


In [123]:
multiply.invoke({'a':2, 'b':3})

6

In [124]:
# now binding this tool to the LLM
# llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)
# Bind tools to the model
llm_with_tool = llm.bind_tools([multiply])

In [125]:
llm_with_tool.invoke('multiply 2 and 3')

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-2ce1139b-dc1d-4ed7-b44d-f6a3b99d4da1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': '98370824-d185-4be2-8c86-1aeaac369f8d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 19, 'output_tokens': 5, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})

In [126]:
t = llm_with_tool.invoke("can you multiply 2 and 3?").tool_calls[0]   #her you will this will return the tool calls made by the model not actually executing them


In [127]:
# now how to execute the tool calls
multiply.invoke(t)
# now result of this i can give to llm to get the final answer

ToolMessage(content='6', name='multiply', tool_call_id='4c9d3471-73d5-402d-8925-f22124295d3a')

## # ----------------------now organize all 

In [148]:

query = HumanMessage('can you multiply 2 and 3?')
messages = [query]

In [149]:
AI_mesasge = llm_with_tool.invoke(messages)  
AI_mesasge
messages.append(AI_mesasge)

In [150]:
tool_message = multiply.invoke(t)
messages.append(tool_message)

In [151]:
messages

[HumanMessage(content='can you multiply 2 and 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-c60d0892-7b27-401a-a7d8-7fc88d2e125e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': 'fbe63113-9252-4a9c-bf5b-d186cc5ce3e0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 22, 'output_tokens': 5, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='6', name='multiply', tool_call_id='4c9d3471-73d5-402d-8925-f22124295d3a')]

In [ ]:
llm_with_tool.invoke(messages)
# llm_with_tool.invoke("hi how are you?")
# str(messages)   042daa57-ac5d-476d-87b2-244a20d7a7ef

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents[2].parts: contents.parts must not be empty.
